In [1]:
!pip install qiskit qiskit_aer numpy matplotlib qiskit-ibm-runtime

In [30]:
# Built-in modules

from qiskit.circuit.library import MCXGate
from qiskit import QuantumCircuit, assemble, QuantumRegister, ClassicalRegister,transpile
from qiskit.visualization import plot_histogram, plot_bloch_vector,plot_bloch_multivector,plot_state_city,plot_distribution
from qiskit_aer import AerSimulator
from numpy.random import randint
import numpy as np

In [3]:
qc = QuantumCircuit(1, 1)
# Alice prepares a qubit |+>
qc.h(0)
qc.barrier()

CircuitInstruction(operation=Instruction(name='barrier', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(1, 'q'), 0),), clbits=())

In [4]:
qc.draw()

┌───┐ ░ 
  q: ┤ H ├─░─
     └───┘ ░ 
c: 1/════════

In [5]:
qc.h(0)
qc.measure(0,0)
qc.draw()

┌───┐ ░ ┌───┐┌─┐
  q: ┤ H ├─░─┤ H ├┤M├
     └───┘ ░ └───┘└╥┘
c: 1/══════════════╩═
                   0

# QKD

In [53]:
np.random.seed(seed=0)
message_length = 100 
#Alice generates a random set of bits
alice_bits = randint(2, size=message_length)

In [54]:
alice_bases = randint(2, size= message_length)

In [55]:
def encode_message(bits, bases):
    message = []
    for i in range(message_length):
        qc = QuantumCircuit(1, 1)
        if bases[i] == 0: # encode in Z - basis
            if bits[i] == 0:
                pass
            else:
                qc.x(0)
        else:
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        qc.barrier()
        message.append(qc)
    return message

In [56]:
message = encode_message(alice_bits, alice_bases)

In [57]:
alice_bits

array([0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0])

In [58]:
alice_bases

array([1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0])

In [59]:
message[0].draw()

┌───┐ ░ 
  q: ┤ H ├─░─
     └───┘ ░ 
c: 1/════════

In [60]:
message[2].draw()

┌───┐ ░ 
  q: ┤ X ├─░─
     └───┘ ░ 
c: 1/════════

In [61]:
eve_bases = randint(2, size= message_length)

In [62]:
def measure_message(message, bases):
    measurements = []
    for i in range(message_length):
        if bases[i] == 0: #if Z basis
            message[i].measure(0,0)
        if bases[i] == 1: # if X basis
            message[i].h(0)
            message[i].measure(0,0)
        simulator = AerSimulator()
        qcirc= transpile(message[i], simulator)
        result = simulator.run(qcirc, shots = 1, memory=True).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

In [63]:
intercepted_message = measure_message(message, eve_bases)

In [66]:
print(intercepted_message)

[0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1]


In [67]:
bob_bases = randint(2, size=message_length)

In [68]:
bob_bases

array([1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1])

In [69]:
bob_results = measure_message(message, bob_bases)

In [70]:
message[2].draw()

┌───┐ ░ ┌───┐┌─┐┌───┐┌─┐
  q: ┤ X ├─░─┤ H ├┤M├┤ H ├┤M├
     └───┘ ░ └───┘└╥┘└───┘└╥┘
c: 1/══════════════╩═══════╩═
                   0       0

In [71]:
def remove_garbage(a_bases, b_bases, bits):
    good_bits = []
    for i in range(message_length):
        if a_bases[i] == b_bases[i]:
            good_bits.append(int(bits[i]))
    return good_bits

In [72]:
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)

In [73]:
bob_key = remove_garbage(alice_bases, bob_bases, bob_results)

In [74]:
print(f"Bob's key {bob_key}\n Alice's key {alice_key}")

Bob's key [1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0]
 Alice's key [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1]


In [75]:
alice_key == bob_key

False

In [76]:
len(alice_key)

47

In [77]:
def sample_bits(bits, selection):
    sample = []
    for i in selection:
        i = np.mod(i, len(bits))
        sample.append(bits.pop(i))
    return sample

In [78]:
sample_size =20
bits_selection = randint(2, size = sample_size)
bob_sample = sample_bits(bob_key, bits_selection)
alice_sample = sample_bits(alice_key, bits_selection)
print(f"Bob's sample: {bob_sample}\n Alice's sample: {alice_sample}")

Bob's sample: [1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1]
 Alice's sample: [0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]


In [79]:
bob_sample == alice_sample

False